In [1]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
import glob
import pandas as pd
import numpy as np
import re
import plotly_express as px
import datetime
import warnings
import os
import json
import streamlit as st

pd.options.display.float_format = "{:,.1f}".format

warnings.filterwarnings('ignore')

global teamnamedict
# C:\Users\mg\JupyterLabDir\Rest\Pr Winning\teamnamedict_streamlit.json
with open('teamnamedict_streamlit.json') as f:
    teamnamedict = json.load(f)

# def _max_width_():
#     max_width_str = f"max-width: 1300px;"
#     st.markdown(
#         f"""
#     <style>
#     .reportview-container .main .block-container{{
#         {max_width_str}
#     }}
#     </style>
#     """,
#         unsafe_allow_html=True,
#     )

# _max_width_()
global widthfig
widthfig = 700
heightfig = 500

# get all the gamestatistics from in dropdown specified league and season
# setup the database connection.  There's no need to setup cursors with pandas psql.
tables = list(glob.glob("htdatan/*"))

# take only the 0 part of the every list entry
global saissons
saissons = []

# ENV is BTCPRED
for x in range(0, len(tables)):    # CHANGE THIS - \\ - to - / - FOR DEPLOYMENT!
    saissons.append(tables[x].split("\\")[1].split("_24102021.csv")[0])


cleaned_names_saissons = []
for saisson in saissons:
    saisson = saisson.replace("_", " ")
    saisson = saisson.replace(".csv", "")
    # saison = saison.strip()
    cleaned_names_saissons.append(saisson)

# map league shortcuts to real names:
shortcut_league_dict = {
    "ll": "La-Liga",
    "pl": "Premier-League",
    "sa": "Serie-A",
    "pl": "Premier-League",
    "l1": "League-1",
    "b": "Bundesliga",
}
# list(map(cleaned_names_saissons, shortcut_league_dict) )
cleaned_names_saissons = [e.replace(
    key, val) for e in cleaned_names_saissons for key, val in shortcut_league_dict.items() if key in e]

# sort list by int substring
cleaned_names_saissons = sorted(cleaned_names_saissons, key=lambda x: int(
    x.split(" ")[-1]), reverse=True)

global saison
saison = st.sidebar.selectbox("Saison", list(
    cleaned_names_saissons), 3)
print(saison)
# map names back for reading the correct csv name


2022-02-12 13:02:26.222 
  command:

    streamlit run C:\Users\markus.gebele\Anaconda3\envs\btcpred\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


Premier-League 2122


In [2]:
def find_key(input_dict, value):
    for key, val in input_dict.items():
        if val == value:
            return key
    return "None"

saison = "{}_{}".format(find_key(shortcut_league_dict, saison.split(" ")[0]),
                        saison.split(" ")[1]
                        )

try:
    df_complete_saison = pd.read_csv(
        "htdatan/"+saison+".csv", index_col=0, encoding='utf-8')
except:
    df_complete_saison = pd.read_csv(
        "htdatan/"+saison+"_24102021.csv", index_col=0, encoding='utf-8')

df_complete_saison = df_complete_saison.replace(teamnamedict)
dfallteamnamesl = df_complete_saison.H_Teamnames.unique()

# take only the 0 part of the every list entry
teamsoptions = []
for x in range(0, len(dfallteamnamesl)):
    teamsoptions.append(dfallteamnamesl[x])

global xg_team
xg_team = st.sidebar.selectbox("Team", list(np.sort(teamsoptions)), 12)

# convert string to df to use process_team_names_of_df function
df_teamname = pd.DataFrame([xg_team])

# convert xg teamnames to correct ones that are used in htdatan
team_df = df_teamname.replace(teamnamedict)
# teamname corrected that it fits to htdatan teamnames
team = team_df.iloc[0][0]

df = df_complete_saison[(df_complete_saison.H_Teamnames == team) | (
    df_complete_saison.A_Teamnames == team)]

# return df, team, saison


In [3]:
teamsoptions

['Brentford',
 'Manchester United',
 'Watford',
 'Leicester',
 'Everton',
 'Chelsea',
 'Burnley',
 'Norwich',
 'Newcastle United',
 'Tottenham',
 'Liverpool',
 'Manchester City',
 'Leeds',
 'Crystal Palace',
 'Aston Villa',
 'Brighton',
 'Wolves',
 'Southampton',
 'Arsenal',
 'West Ham']

In [4]:
compare_team = teamsoptions[11]
compare_team

'Manchester City'

In [5]:
# convert string to df to use process_team_names_of_df function
df_teamname = pd.DataFrame([xg_team])

# convert xg teamnames to correct ones that are used in htdatan
team_df = df_teamname.replace(teamnamedict)
# teamname corrected that it fits to htdatan teamnames
team = team_df.iloc[0][0]

df = df_complete_saison[(df_complete_saison.H_Teamnames == team) | (
    df_complete_saison.A_Teamnames == team)]

# return df, team, saison

In [6]:
def process_team_names_of_df(x_df):
    x_df = x_df.replace(teamnamedict)
    return x_df

#######################################################
###  calculate table with two halftimes to one game ###
#######################################################

def convert_hts_to_complete_games(df):
    # fill nane values of these not numeric columns
    df[['FK-H', 'FK-A']].fillna(0)
    # convert not numeric columns to numeric columns
    df['FK-H'] = df['FK-H'].astype('float64')
    df['FK-A'] = df['FK-A'].astype('float64')
    df['C-H'] = df['C-H'].astype('float64')
    df['F-H'] = df['F-H'].astype('float64')
    df['GA-H'] = df['GA-H'].astype('float64')
    df['SoffG-H'] = df['SoffG-H'].astype('float64')
    df['SoG-H'] = df['SoG-H'].astype('float64')
    df['C-A'] = df['C-A'].astype('float64')
    df['F-A'] = df['F-A'].astype('float64')
    df['GA-A'] = df['GA-A'].astype('float64')
    df['SoffG-A'] = df['SoffG-A'].astype('float64')
    df['SoG-A'] = df['SoG-A'].astype('float64')
    df['G-H'] = df['G-H'].astype('float64')
    df['G-A'] = df['G-A'].astype('float64')
    df['BP-H'] = df['BP-H'].astype('float64')
    df['BP-A'] = df['BP-A'].astype('float64')
    # xGoals columns
    if not set(['xG', 'xPTS', 'GOALS', 'A_xG', 'G-A', 'A_xPTS']).issubset(df.columns):
        df['xG'] = -1.0
        df['GOALS'] = -1.0
        df['xPTS'] = -1.0
        df['A_xG'] = -1.0
        df['G-A'] = -1.0
        df['A_xPTS'] = -1.0
    else:
        df['xG'] = df['xG'].astype('float64')
        df['GOALS'] = df['GOALS'].astype('float64')
        df['xPTS'] = df['xPTS'].astype('float64')
        df['A_xG'] = df['A_xG'].astype('float64')
        df['G-A'] = df['G-A'].astype('float64')
        df['A_xPTS'] = df['A_xPTS'].astype('float64')

    # calculate halftime table to fulltime table
    df = df.groupby(['Home', 'Opponent', 'Date', 'Round']).agg({'BP-H': 'mean', 'C-H': 'sum',
                                                                'F-H': 'sum', 'FK-H': 'sum', 'GA-H': 'sum',
                                                                'GoKeSa-H': 'sum', 'G-H': 'sum', 'Off-H': 'sum',
                                                                'SoffG-H': 'sum', 'SoG-H': 'sum',
                                                                           'BP-A': 'mean',
                                                                           'C-A': 'sum',
                                                                           'F-A': 'sum', 'FK-A': 'sum', 'GA-A': 'sum',
                                                                           'GoKeSa-A': 'sum', 'G-A': 'sum', 'Off-A': 'sum',
                                                                           'SoffG-A': 'sum', 'SoG-A': 'sum',
                                                                           # xGoals stats are only from whole game - not halftime - so mean does not change anything
                                                                           'xG': 'mean',
                                                                           'GOALS': 'mean',
                                                                           'xPTS': 'mean',
                                                                           'A_xG': 'mean',
                                                                           'A_GOALS': 'mean',
                                                                           'A_xPTS': 'mean'
                                                                }).reset_index()

    newcols = []

    for x in df.columns:
        if x.startswith('SUM') or x.startswith('MIN') or x.startswith('AVG'):
            x = re.sub('SUM', '', x)
            x = re.sub('MIN', '', x)
            x = re.sub('AVG', '', x)
            x = x.replace("`", "")
            x = x.replace(")", "")
            x = x.replace("(", "")
        newcols.append(x)

    df.columns = newcols

    return df


def df_cleaning_converting(df):
    df = df[['H_Teamnames', 'A_Teamnames', 'H_Goals', 'A_Goals', 'H_Ball Possession', 'A_Ball Possession', 'A_Goal Attempts', 'H_Goal Attempts',
             'H_Shots on Goal', 'A_Shots on Goal', 'H_Shots off Goal', 'A_Shots off Goal', 'H_Free Kicks',
             'A_Free Kicks', 'H_Corner Kicks', 'A_Corner Kicks', 'H_Offsides', 'A_Offsides', 'H_Goalkeeper Saves', 'A_Goalkeeper Saves',
             'H_Fouls', 'A_Fouls', 'A_gameinfo', 'A_datetime', 'xG', 'GOALS', 'xPTS', 'A_xG', 'A_GOALS', 'A_xPTS']]

    df["R"] = 'X'

    for i in range(0, len(df)):
        try:

            if df["H_Goals"][i] > df["A_Goals"][i]:
                df["R"][i] = 'H'
            if df["H_Goals"][i] < df["A_Goals"][i]:
                df["R"][i] = 'A'
            else:
                df["R"][i] = 'D'
        except:
            print("error?")

    df.columns = ['Home', 'Opponent', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
                  'SoG-H', 'SoG-A', 'SoffG-H', 'SoffG-A', 'FK-H',
                  'FK-A', 'C-H', 'C-A', 'Off-H', 'Off-A', 'GoKeSa-H', 'GoKeSa-A',
                  'F-H', 'F-A', 'Round', 'Date', 'xG', 'GOALS', 'xPTS', 'A_xG', 'A_GOALS', 'A_xPTS', 'R']

    df = df[['Home', 'Opponent', 'R', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
             'SoG-H', 'SoG-A', 'SoffG-H', 'SoffG-A', 'FK-H',
             'FK-A', 'C-H', 'C-A', 'Off-H', 'Off-A', 'GoKeSa-H', 'GoKeSa-A', 'F-H',
             'F-A', 'Round', 'Date', 'xG', 'GOALS', 'xPTS', 'A_xG', 'A_GOALS', 'A_xPTS']]

    df["IsHome"] = 0

    df = df[['Home', 'Opponent', 'R', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
             'SoG-H', 'SoG-A', 'SoffG-H', 'SoffG-A', 'FK-H',
             'FK-A', 'C-H', 'C-A', 'Off-H', 'Off-A', 'GoKeSa-H', 'GoKeSa-A', 'F-H',
             'F-A', 'Round', 'Date', 'IsHome', 'xG', 'GOALS', 'xPTS', 'A_xG', 'A_GOALS', 'A_xPTS']]
    return df


def df_specific_team(df, team):
    df4Home = df.loc[((df['Home'] == team))]

    df4Home["IsHome"] = 1

    # recalcualte the winner because of the columns switching to bring the selected team in the first column
    df4Home["1x2"] = 0

    df4Home["1x2"] = df4Home.apply(
        lambda row: calculate_1x2_home(row), axis=1, result_type='reduce')

    # Berechnung Opponentgames
    df4Opponent = df.loc[((df['Opponent'] == team))]

    # recalcualte the winner because of the columns switching to bring the selected team in the first column
    df4Opponent["1x2"] = 0

    df4Opponent["1x2"] = df4Opponent.apply(
        lambda row: calculate_1x2_Opponent(row), axis=1, result_type='reduce')

    # Change the columns for the Opponentmatches of the specific team
    OpponentTeamReversedColumns = ['Opponent', 'Home',  '1x2', 'R',  'G-A', 'G-H',
                                   'BP-A', 'BP-H', 'GA-A', 'GA-H',  'SoG-A', 'SoG-H', 'SoffG-A', 'SoffG-H',  'FK-A', 'FK-H',
                                   'C-A', 'C-H',  'Off-A', 'Off-H', 'GoKeSa-A', 'GoKeSa-H', 'F-A', 'F-H',
                                   'Round', 'Date', 'IsHome',
                                   'A_xG', "A_xPTS", "A_GOALS", 'xG', "xPTS", "GOALS", ]  # , 'IsHome'

    df4OpponentReversed = df4Opponent.reindex(
        columns=OpponentTeamReversedColumns)

    df4OpponentReversed.columns = ['Home', 'Opponent', '1x2', 'R', 'G-H',
                                   'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
                                   'SoG-H', 'SoG-A', 'SoffG-H', 'SoffG-A', 'FK-H',
                                   'FK-A', 'C-H', 'C-A', 'Off-H', 'Off-A', 'GoKeSa-H', 'GoKeSa-A', 'F-H',
                                   'F-A', 'Round', 'Date', 'IsHome',
                                   'xG', "xPTS", "GOALS", 'A_xG', "A_xPTS", "A_GOALS"]
    return df4Home, df4OpponentReversed


def create_df4Complete(df4Home, df4OpponentReversed):
    # Alle Spiele werden als Heimspiel angezeigt, sind aber auch Auswärtsspiele dabei!
    df4Complete = pd.concat([df4Home, df4OpponentReversed], sort=False)

    df4Complete["G-H"] = df4Complete["G-H"].astype('float64')
    df4Complete["G-A"] = df4Complete["G-A"].astype('float64')
    df4Complete["BP-H"] = df4Complete["BP-H"].astype('float64')
    df4Complete["BP-A"] = df4Complete["BP-A"].astype('float64')

    # GoalDifference
    df4Complete["GoalDiff"] = df4Complete["G-H"] - df4Complete["G-A"]
    df4Complete = df4Complete.sort_values("Date",  ascending=False)

    # calculate column with 3 Ballposition types
    df4Complete["BPTypes"] = '0'

    df4Complete["BPTypes"] = df4Complete.apply(
        lambda row: calculate_1x2_BPTypes(row), axis=1, result_type='reduce')

    df4Complete['Date'] = pd.to_datetime(
        df4Complete['Date'], format="%d.%m.%Y %H:%M")

    # convert datetime to timestamp for scatter visualization
    df4Complete['timestamp'] = df4Complete.Date.astype('int64')//10**9

    df4Complete = df4Complete.sort_values("Date", ascending=False)

    # Create data for scatter graph
    df4Complete["SoG-H"] = df4Complete["SoG-H"].astype(int)
    df4Complete["SoG-A"] = df4Complete["SoG-A"].astype(int)

    return df4Complete


def calculate_1x2_home(row):
    if row['G-H'] > row['G-A']:
        return 'W'
    elif row['G-A'] > row['G-H']:
        return 'L'
    else:
        return 'D'


def calculate_1x2_Opponent(row):
    if row['G-A'] > row['G-H']:
        return 'W'
    elif row['G-H'] > row['G-A']:
        return 'L'
    else:
        return 'D'


def calculate_1x2_BPTypes(row):
    if row['BP-H'] > 55:
        return '>55'
    elif row['BP-H'] < 45:
        return '<45'
    else:
        return '45-55'


def calc_stats(df4Complete):
    BP_WPerc = df4Complete[['BPTypes', '1x2']
                           ].loc[df4Complete['BPTypes'] == '>55']
    BP_WAbs = BP_WPerc['1x2'].loc[df4Complete['1x2'] == 'W']
    BP_NWAbs = BP_WPerc['1x2'].loc[df4Complete['1x2'] != 'W']
    if len(BP_WAbs) + len(BP_NWAbs) > 0:
        BP_WPercText = len(BP_WAbs) / (len(BP_WAbs) + len(BP_NWAbs)) * 100
        BP_WPercText = round(BP_WPercText)
    else:
        BP_WPercText = 0

    # calculate winning % for 0.45 - 0.55
    N_WPerc = df4Complete[['BPTypes', '1x2']
                          ].loc[df4Complete['BPTypes'] == '45-55']
    N_WAbs = N_WPerc['1x2'].loc[df4Complete['1x2'] == 'W']
    N_NWAbs = N_WPerc['1x2'].loc[df4Complete['1x2'] != 'W']
    if len(N_WAbs) + len(N_NWAbs) > 0:
        N_WPercText = len(N_WAbs) / (len(N_WAbs) + len(N_NWAbs)) * 100
        N_WPercText = round(N_WPercText)
    else:
        N_WPercText = 5

    # calculate winning % for < 0.45
    C_WPerc = df4Complete[['BPTypes', '1x2']
                          ].loc[df4Complete['BPTypes'] == '<45']
    C_WAbs = C_WPerc['1x2'].loc[df4Complete['1x2'] == 'W']
    C_NWAbs = C_WPerc['1x2'].loc[df4Complete['1x2'] != 'W']
    if len(C_WAbs) + len(C_NWAbs) > 0:
        C_WPercText = len(C_WAbs) / (len(C_WAbs) + len(C_NWAbs)) * 100
        C_WPercText = round(C_WPercText)
    else:
        C_WPercText = 0

    return C_WPercText, N_WPercText, BP_WPercText


# get name of the selected team in dropdown
def load_xg_gamestats_sql(saison, team):

    if saison.split("_")[0] == 'b':
        xgprefix = 'bundesliga'
    elif saison.split("_")[0] == 'l1':
        xgprefix = 'ligue_1'
    elif saison.split("_")[0] == 'll':
        xgprefix = 'la_liga'
    elif saison.split("_")[0] == 'pl':
        xgprefix = 'epl'
    elif saison.split("_")[0] == 'sa':
        xgprefix = 'serie_a'

    xgtablename = "{}20{}".format(xgprefix, saison.split("_")[1][:2])

    df_complete_saison = pd.read_csv(
        "xg/"+xgtablename+".csv", index_col=0, encoding='utf-8')

    # execute the query and assign it to a pandas dataframe
    dfxg = df_complete_saison[(df_complete_saison.TEAMS == team) | (
        df_complete_saison.A_TEAMS == team)]

    dfxg = process_team_names_of_df(dfxg)

    return dfxg

In [28]:
df4CompleteGraph = df4CompleteGraph[df4CompleteGraph.groupby(
    'Opponent')['Opponent'].transform('size') >= 2]
df4CompleteGraph = df4CompleteGraph.sort_index()
# second half is second entry always!
df4CompleteGraph["halftime"] = "0"
df4CompleteGraph.iloc[::2]["halftime"] = "2"
df4CompleteGraph.iloc[1::2]["halftime"] = "1"

In [34]:
df4CompleteGraph.columns

Index(['Home', 'Opponent', 'Date', 'Round', 'BP-H', 'C-H', 'F-H', 'FK-H',
       'GA-H', 'GoKeSa-H', 'G-H', 'Off-H', 'SoffG-H', 'SoG-H', 'BP-A', 'C-A',
       'F-A', 'FK-A', 'GA-A', 'GoKeSa-A', 'G-A', 'Off-A', 'SoffG-A', 'SoG-A',
       'xG', 'GOALS', 'xPTS', 'A_xG', 'A_GOALS', 'A_xPTS', 'GoalDiff',
       'BPTypes', 'timestamp', 'xG-A_xG', 'A_xG-xG', 'halftime'],
      dtype='object')

In [43]:
df4CompleteGraph.groupby(["Opponent","halftime","Halftime result"]).agg('count').sort_values("Opponent", ascending=False).iloc[0].Home

4

In [44]:
df4CompleteGraph[df4CompleteGraph["halftime"] == "2"]

,Home,Opponent,Date,Round,BP-H,C-H,F-H,FK-H,GA-H,GoKeSa-H,...,xPTS,A_xG,A_GOALS,A_xPTS,GoalDiff,BPTypes,timestamp,xG-A_xG,A_xG-xG,halftime
1,Manchester United,Aston Villa,2021-09-25 04:30:00,FootballFootballENGLAND: Premier League - Round 6,59.0,5.0,9.0,14.0,7.0,2,...,1.6,1.6,1.0,1.1,-1.0,>55,1632544200,0.3,0.0,2
12,Manchester United,Newcastle United,2021-09-11 07:00:00,FootballFootballENGLAND: Premier League - Round 4,64.0,7.0,9.0,5.0,12.0,2,...,2.8,0.6,1.0,0.2,3.0,>55,1631343600,2.2,0.0,2
18,Manchester United,West Ham,2021-09-19 06:00:00,FootballFootballENGLAND: Premier League - Round 5,60.0,11.0,8.0,7.0,13.0,3,...,1.5,1.7,1.0,1.2,1.0,>55,1632031200,0.2,0.0,2
20,Manchester United,Wolves,2021-08-29 08:30:00,FootballFootballENGLAND: Premier League - Round 3,56.0,7.0,9.0,12.0,15.0,6,...,0.7,1.8,0.0,2.1,1.0,>55,1630225800,0.0,0.9,2


In [7]:
df = process_team_names_of_df(df)
df

,H_datetime,H_gameinfo,H_Teamnames,H_Goals,H_Ball Possession,H_Goal Attempts,H_Shots on Goal,H_Shots off Goal,H_Blocked Shots,H_Free Kicks,...,A_Fouls,A_Yellow Cards,A_Red Cards,A_Total Passes,A_Completed Passes,A_Tackles,A_Attacks,A_Dangerous Attacks,A_Throw-in,H_cat
2,14.08.2021 04:30,FootballFootballENGLAND: Premier League - Round 1,Manchester United,4,52,5,4,1,0,7,...,6,2.0,NaN,209,155,10,48,14,11,cat2
3,14.08.2021 04:30,FootballFootballENGLAND: Premier League - Round 1,Manchester United,1,46,11,4,4,3,5,...,3,NaN,NaN,229,178,13,43,21,6,cat3
34,22.08.2021 06:00,FootballFootballENGLAND: Premier League - Round 2,Southampton,0,37,4,2,2,0,6,...,4,2.0,NaN,264,210,9,58,25,8,cat34
35,22.08.2021 06:00,FootballFootballENGLAND: Premier League - Round 2,Southampton,1,35,4,1,2,1,7,...,6,1.0,NaN,256,204,10,47,25,10,cat35
58,29.08.2021 08:30,FootballFootballENGLAND: Premier League - Round 3,Wolves,0,44,6,3,3,0,6,...,4,3.0,NaN,281,240,4,56,32,8,cat58
59,29.08.2021 08:30,FootballFootballENGLAND: Premier League - Round 3,Wolves,0,44,9,3,3,3,7,...,5,1.0,NaN,271,238,4,62,12,6,cat59
66,11.09.2021 07:00,FootballFootballENGLAND: Premier League - Round 4,Manchester United,3,58,6,4,1,1,2,...,0,0.0,NaN,221,181,8,35,9,6,cat66
67,11.09.2021 07:00,FootballFootballENGLAND: Premier League - Round 4,Manchester United,1,70,15,3,8,4,3,...,2,NaN,NaN,144,106,9,37,18,13,cat67
94,19.09.2021 06:00,FootballFootballENGLAND: Premier League - Round 5,West Ham,0,44,7,2,2,3,5,...,6,NaN,NaN,294,248,8,73,32,8,cat94
95,19.09.2021 06:00,FootballFootballENGLAND: Premier League - Round 5,West Ham,1,36,6,2,3,1,2,...,2,NaN,NaN,330,307,6,61,43,5,cat95


In [8]:
team

'Manchester United'

In [9]:
dfxg = load_xg_gamestats_sql(saison, team)

# rename columns for
dfxg_rename = dfxg.rename(
    columns={'TEAMS': 'H_Teamnames', 'A_TEAMS': 'A_Teamnames'})
# del dfxg

dfxg_df_merged = pd.merge(
    df, dfxg_rename, on=["H_Teamnames", "A_Teamnames"])
dfxg_df_merged = dfxg_df_merged.drop_duplicates()

# %%
df = dfxg_df_merged

dfxg_df_merged_cleaned = df_cleaning_converting(df)

df4Home, df4OpponentReversed = df_specific_team(
    dfxg_df_merged_cleaned, team)

df4Complete = create_df4Complete(df4Home, df4OpponentReversed)

In [10]:
df4Complete

,Home,Opponent,R,G-H,G-A,BP-H,BP-A,GA-H,GA-A,SoG-H,...,xG,GOALS,xPTS,A_xG,A_GOALS,A_xPTS,1x2,GoalDiff,BPTypes,timestamp
42,Manchester United,West Ham,D,1.0,0.0,59.0,41.0,5,13,3,...,1.8,1,2.7,0.2,0,0.1,W,1.0,>55,1642867200
43,Manchester United,West Ham,D,0.0,0.0,55.0,45.0,1,5,0,...,1.8,1,2.7,0.2,0,0.1,D,0.0,45-55,1642867200
40,Manchester United,Brentford,A,3.0,1.0,46.0,54.0,10,9,5,...,2.1,3,1.4,2.1,1,1.4,W,2.0,45-55,1642626000
41,Manchester United,Brentford,D,0.0,0.0,68.0,32.0,8,4,0,...,2.1,3,1.4,2.1,1,1.4,D,0.0,>55,1642626000
38,Manchester United,Aston Villa,D,1.0,2.0,35.0,65.0,9,6,2,...,1.4,2,1.0,1.8,2,1.8,L,-1.0,<45,1642271400
39,Manchester United,Aston Villa,A,1.0,0.0,55.0,45.0,4,7,4,...,1.4,2,1.0,1.8,2,1.8,W,1.0,45-55,1642271400
36,Manchester United,Wolves,A,0.0,1.0,56.0,44.0,5,5,1,...,0.8,0,1.2,0.8,1,1.4,L,-1.0,>55,1641234600
37,Manchester United,Wolves,D,0.0,0.0,46.0,54.0,14,4,1,...,0.8,0,1.2,0.8,1,1.4,D,0.0,45-55,1641234600
34,Manchester United,Burnley,D,0.0,0.0,57.0,43.0,4,6,2,...,1.8,3,2.1,0.9,1,0.7,D,0.0,>55,1640898900
35,Manchester United,Burnley,D,3.0,1.0,57.0,43.0,6,12,4,...,1.8,3,2.1,0.9,1,0.7,W,2.0,>55,1640898900


In [11]:
import plotly.graph_objects as go

In [12]:
slidertext = 'Show last x halftimes'
nrGames = st.sidebar.slider(slidertext, max_value=len(
    df4Complete), value=len(df4Complete), step=2)

# change rows of df depending on userinput
df4Complete = df4Complete[:nrGames]
df4Complete = df4Complete.sort_values("Date", ascending=False)
df4Complete = df4Complete.round(1)

df4Complete[['xG', 'A_xG', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A', 'SoG-H', 'SoG-A',  'xPTS', 'A_xPTS']] = df4Complete[['xG',
                                                                                                                              'A_xG', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A', 'SoG-H', 'SoG-A',  'xPTS', 'A_xPTS']].apply(pd.to_numeric, errors='coerce', axis=1)

df4Complete_show = df4Complete[['Home', 'Opponent', 'IsHome', 'R', 'xG', 'A_xG', 'G-H', 'G-A', 'BP-H', 'BP-A', 'GA-H', 'GA-A',
                                'SoG-H', 'SoG-A',  'xPTS', 'A_xPTS', 'Date']]


# create df for visualizing
df4CompleteGraph = df4Complete.copy()

df4CompleteGraph["SoG-H-SoG-A"] = df4CompleteGraph["SoG-H"] - \
    df4CompleteGraph["SoG-A"]
df4CompleteGraph["SoG-H-SoG-A"] = df4CompleteGraph["SoG-H-SoG-A"].clip(
    lower=0)

df4CompleteGraph["SoG-A-SoG-H"] = df4CompleteGraph["SoG-A"] - \
    df4CompleteGraph["SoG-H"]
df4CompleteGraph["SoG-A-SoG-H"] = df4CompleteGraph["SoG-A-SoG-H"].clip(
    lower=0)

figScatter = go.Figure(data=go.Scatter(x=df4CompleteGraph['BP-H'],
    y=df4CompleteGraph['GoalDiff'], mode='markers', ))

# figScatter = px.scatter(
#     df4CompleteGraph,  # .query(f'Date.between{end_date}'),
#     x=df4CompleteGraph['BP-H'],
#     y=df4CompleteGraph['GoalDiff'],
#     color="timestamp",
#     size="SoG-H-SoG-A",
#     text="Opponent",
#     width=widthfig,
#     # height=heightfig,
#     # title="SoGH-SoGA - Halftimes",
#     # color_continuous_scale= 'Viridis',
#     # color_discrete_map={"W": "green", "D": "gray", "L": "red"}

#     # facet_row="time", # makes seperate plot for value
#     # a="histogram",
# ).update_traces(textposition='middle right')
# figScatter.update_xaxes(range=[5, 95])
# figScatter.update_layout(
#     title_text='Halftimes: Shots on Goal - Shots on Goal Opponent', title_x=0.5,
#     yaxis=dict(
#         tickmode='linear',
#         tick0=1,
#         dtick=1,
#         title="Goal difference"
#     ))

# %%
figScatter

In [13]:
# fig_hist = px.histogram(df4CompleteGraph, x="", width=widthfig, nbins=20)
# fig_hist.show()

figHistogram = go.Figure(data=[go.Histogram(x=df4CompleteGraph["BP-H"], nbinsx=20)])
figHistogram.update_xaxes(range=[5, 95])
figHistogram.show()

In [14]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=1, row_width=[0.3, 0.7])

fig.append_trace(go.Scatter(x=df4CompleteGraph['BP-H'],
    y=df4CompleteGraph['GoalDiff'], mode='markers', ), row=1, col=1)

fig.append_trace(go.Histogram(x=df4CompleteGraph["BP-H"], nbinsx=20), row=2, col=1)

fig.update_layout(height=600, width=600, title_text="Stacked Subplots")
fig.show()

In [15]:
# wie oft passierts das team hinten ist und noch gewinnt / nicht verliert:´
# sortier nach sieg x loss 
#mach diagramm halftimes für 1te und 2te hz

# delete games where there is no two halftimes!
df4Complete = df4CompleteGraph[df4CompleteGraph.groupby('Opponent')['Opponent'].transform('size') >= 2]
df4Complete = df4Complete.sort_index()
# second half is second entry always!
df4Complete["halftime"] = 0
df4Complete.iloc[::2]["halftime"] = 2
df4Complete.iloc[1::2]["halftime"] = 1

In [16]:

figScatter = px.scatter(
    df4Complete,  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    color="halftime",
    size="SoG-H-SoG-A",
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # title="SoGH-SoGA - Halftimes",
    # color_continuous_scale= 'Viridis',
    # color_discrete_map={"W": "green", "D": "gray", "L": "red"}

    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(textposition='top center')
figScatter.update_xaxes(range=[5, 95])
figScatter.update_layout(
    title_text='Halftimes: Shots on Goal - Shots on Goal Opponent', title_x=0.5,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))

# %%

In [17]:
figScatter1 = px.scatter(
    df4CompleteGraph,  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    color="timestamp",
    size="SoG-A-SoG-H",
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    title="SoGA-SoGH - Halftimes",
    # color_discrete_map={"W": "green", "D": "gray", "L": "red"}

    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(textposition='top center')
figScatter1.update_xaxes(range=[5, 95])
figScatter1.update_layout(
    title_text='Halftimes: Shots on Goal Opponent - Shots on Goal', title_x=0.5,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    )
)

naming1x2 = {"W": "Win", "D": "Draw", "L": "Loss"}
df4CompleteGraph['Halftime result'] = df4CompleteGraph['1x2'].replace(
    naming1x2)
# Create data for histogram 2
figHist2 = px.bar(
    df4CompleteGraph,
    x='BPTypes',
    # text=df4CompleteGraph.index,
    # title="BP-Styles - Halftimes",
    color='Halftime result',
    color_discrete_map={"Win": "green", "Draw": "gray", "Loss": "red"},
    width=widthfig,
    # height=heightfig,
    # opacity=0.5,
    # text="Opponent",
).update_xaxes(categoryorder="array",  categoryarray=['<45', '45-55', '>55'])
figHist2.update_layout(
    title_text='Ballpossesionstyles - results per halftime', title_x=0.5, xaxis=dict(
        tickmode='array', showticklabels=True,
    )
)


# create scatterplot with XG - bubble size
df4CompleteGraph = df4Complete.copy()

df4CompleteGraph = convert_hts_to_complete_games(df4CompleteGraph)

# Calculate again the stuff like for the single halftimes before!
# GoalDifference
df4CompleteGraph["GoalDiff"] = df4CompleteGraph["G-H"] - \
    df4CompleteGraph["G-A"]
df4CompleteGraph = df4CompleteGraph.sort_values("Date",  ascending=False)
# calculate column with 3 Ballposition types
df4CompleteGraph["BPTypes"] = '0'
df4CompleteGraph["BPTypes"] = df4CompleteGraph.apply(
    lambda row: calculate_1x2_BPTypes(row), axis=1, result_type='reduce')
df4CompleteGraph['Date'] = pd.to_datetime(
    df4CompleteGraph['Date'], format="%d.%m.%Y %H:%M")
# convert datetime to timestamp for scatter visualization
df4CompleteGraph['timestamp'] = df4CompleteGraph.Date.astype('int64')//10**9
df4CompleteGraph = df4CompleteGraph.sort_values("Date", ascending=False)
# Create data for scatter graph
df4CompleteGraph["xG-A_xG"] = df4CompleteGraph["xG"] - df4CompleteGraph["A_xG"]
df4CompleteGraph["xG-A_xG"] = df4CompleteGraph["xG-A_xG"].clip(lower=0)
df4CompleteGraph["xG-A_xG"] = df4CompleteGraph["xG-A_xG"].round(1)
df4CompleteGraph["A_xG-xG"] = df4CompleteGraph["A_xG"] - df4CompleteGraph["xG"]
df4CompleteGraph["A_xG-xG"] = df4CompleteGraph["A_xG-xG"].clip(lower=0)
df4CompleteGraph["A_xG-xG"] = df4CompleteGraph["A_xG-xG"].round(1)


figScatter3 = px.scatter(
    df4CompleteGraph,  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    color="timestamp",
    size="xG-A_xG",
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # color_continuous_scale= 'Viridis',
    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(textposition='top center', marker_symbol="cross")
figScatter3.update_xaxes(range=[5, 95])
figScatter3.update_layout(
    title_text='Expectedgoals - Expectedgoals Opponent', title_x=0.5,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))

figScatter4 = px.scatter(
    df4CompleteGraph,  # .query(f'Date.between{end_date}'),
    x='BP-H',
    y='GoalDiff',
    color="timestamp",
    size="A_xG-xG",
    text="Opponent",
    width=widthfig,
    # height=heightfig,
    # facet_row="time", # makes seperate plot for value
    # marginal_x="histogram",
).update_traces(textposition='top center')
figScatter4.update_xaxes(range=[5, 95])
figScatter4.update_layout(
    title_text='Expectedgoals Opponent - Expectedgoals', title_x=0.5,
    yaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1,
        title="Goal difference"
    ))


# Streamlit encourages well-structured code, like starting execution in a main() function.
st.title("Football statistics - {}".format(team))

st.markdown('The following two diagrams display the new metric Expected Goals (**xGoals**), which is a qualitative measurement on base of the shots on goal.  \nThe expected goal model shows how high the chance of the goal really was and calculates a value for each completion based on several factors.   \nF.I. a penalty has generally a probably of 75 % to result in a goal, which would increase the xGoal value for 0.75 regardless of the penalty-outcame in this case.', unsafe_allow_html=False)

col1, col2 = st.columns(2)

col1.plotly_chart(figScatter3)

col2.plotly_chart(figScatter4)

col1.plotly_chart(figScatter)

col2.plotly_chart(figScatter1)

col1.plotly_chart(figHist2)


# C_WPercText, N_WPercText, BP_WPercText = calc_stats(df4Complete)
# col2.write("% W < 0.45:   {}   \n % W 0.45 - 0.55:  {}   \n % W > 0.55:  {}".format(
#     C_WPercText, N_WPercText, BP_WPercText))

# show df
print(df4Complete_show.style.format({'xG': '{:.1f}', 'A_xG': '{:.1f}', 'SoG-H': '{:.0f}',
                                            'G-H': '{:.0f}', 'G-A': '{:.0f}', 'BP-H': '{:.0f}',
                                            'BP-A': '{:.0f}', 'GA-H': '{:.0f}', 'GA-A': '{:.0f}',
                                            'xPTS': '{:.1f}', 'A_xPTS': '{:.1f}', 'SoG-A': '{:.0f}',
                                            }))


In [18]:
df4Complete_show.style.format({'xG': '{:.1f}', 'A_xG': '{:.1f}', 'SoG-H': '{:.0f}',
                                            'G-H': '{:.0f}', 'G-A': '{:.0f}', 'BP-H': '{:.0f}',
                                            'BP-A': '{:.0f}', 'GA-H': '{:.0f}', 'GA-A': '{:.0f}',
                                            'xPTS': '{:.1f}', 'A_xPTS': '{:.1f}', 'SoG-A': '{:.0f}',
                                            })

,Home,Opponent,IsHome,R,xG,A_xG,G-H,G-A,BP-H,BP-A,GA-H,GA-A,SoG-H,SoG-A,xPTS,A_xPTS,Date
42,Manchester United,West Ham,1,D,1.8,0.2,1,0,59,41,5,13,3,1,2.7,0.2,2022-01-22 16:00:00
43,Manchester United,West Ham,1,D,1.8,0.2,0,0,55,45,1,5,0,0,2.7,0.2,2022-01-22 16:00:00
40,Manchester United,Brentford,0,A,2.1,2.1,3,1,46,54,10,9,5,5,1.4,1.4,2022-01-19 21:00:00
41,Manchester United,Brentford,0,D,2.1,2.1,0,0,68,32,8,4,0,3,1.4,1.4,2022-01-19 21:00:00
38,Manchester United,Aston Villa,0,D,1.4,1.8,1,2,35,65,9,6,2,6,1.0,1.8,2022-01-15 18:30:00
39,Manchester United,Aston Villa,0,A,1.4,1.8,1,0,55,45,4,7,4,3,1.0,1.8,2022-01-15 18:30:00
36,Manchester United,Wolves,1,A,0.8,0.8,0,1,56,44,5,5,1,2,1.2,1.4,2022-01-03 18:30:00
37,Manchester United,Wolves,1,D,0.8,0.8,0,0,46,54,14,4,1,4,1.2,1.4,2022-01-03 18:30:00
34,Manchester United,Burnley,1,D,1.8,0.9,0,0,57,43,4,6,2,2,2.1,0.7,2021-12-30 21:15:00
35,Manchester United,Burnley,1,D,1.8,0.9,3,1,57,43,6,12,4,1,2.1,0.7,2021-12-30 21:15:00


# add team to compare

In [19]:
# also add last season matches? (if they are not longer than 12 months ago?)
compare_team

'Manchester City'

In [20]:
df4Complete_show[(df4Complete_show["Opponent"]==compare_team)][['Home', 'G-H', 'xG', "BP-H", "BP-A", 'G-A', 'A_xG',  'Opponent',"Date",]]

,Home,G-H,xG,BP-H,BP-A,G-A,A_xG,Opponent,Date
21,Manchester United,0.0,0.8,31.0,69.0,2.0,2.0,Manchester City,2021-11-06 13:30:00
20,Manchester United,0.0,0.8,35.0,65.0,0.0,2.0,Manchester City,2021-11-06 13:30:00


In [21]:
BP-H

NameError: name 'BP' is not defined